In [ ]:

from google.colab import drive
#drive.mount('/content/drive')
import io
import zipfile
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
import librosa
import numpy as np
from scipy.io import wavfile
import pickle


set_size = 4000

dataset = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/audio_train.zip')
datalabels = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
filelist = dataset.namelist()
file10 = filelist[1:]




def read_from_zip(zip_store, filename):
    with io.BytesIO() as bytes_buff:
        bytes_data = zip_store.read(filename)
        bytes_buff.write(bytes_data)
        bytes_buff.seek(0)
        sr, wave = wavfile.read(bytes_buff)
    return sr, wave


m = int(np.max([len(read_from_zip(dataset, file10[i])[1]) for i in range(len(file10))]))
print(m)
#попробовать повторениями дополнить

480000


In [ ]:

#print(datalabels["label"].value_counts())

left_columns = []

for i in datalabels["label"].unique():
  if len(datalabels[datalabels["label"] == i]) > 180:
    left_columns.append(i)

#print(left_columns)

less_data = datalabels.loc[datalabels["label"].isin(left_columns)]
#3102 
less_data.head()

,fname,label
5,2d2900e792da5d4f6ab2.wav,Trumpet
11,a0ba703211ad22d66788.wav,Applause
12,e36e70d42b9ec2dfc283.wav,Shatter
16,fd9a8ca19d270e28dae8.wav,Bass_drum
19,9d51b14bd8d26f9475b9.wav,Bass_drum


In [ ]:
mels = []
labels = less_data["label"].to_numpy()
for i in less_data.iterrows():
  fname = "train/"+i[1]["fname"]
  sr, wave = read_from_zip(dataset, fname)
  if len(wave) < m:
    rep = m // len(wave)
    add = m % len(wave)
    wave = np.ravel(np.concatenate((np.ravel(np.tile(wave, (rep, 1))),wave[:add])))
  mel = np.ravel(librosa.feature.melspectrogram(y=wave,sr = sr))
  mels.append(mel)


  
f = open("train_mels.pickle","wb")
pickle.dump(mels,f)
f.close()

f = open("train_labels.pickle","wb")
pickle.dump(labels,f)
f.close()


#rewrite loop because it may have unnecessary work done

In [ ]:
X = mels
Y = labels
train_x,test_x,train_y,test_y = train_test_split(X,Y,test_size = 0.33)


In [ ]:
print(train_x.shape)
print(train_y.shape)
train_x

(1131, 120064)
(1131,)


array([[5.32300638e-03, 3.56182723e-02, 5.08191339e-02, ...,
        3.67466308e-06, 2.62923033e-06, 2.06820944e-06],
       [2.57130539e-04, 1.10757549e-03, 1.82897813e-03, ...,
        2.95788304e-06, 2.46216616e-06, 2.96184010e-06],
       [3.08105386e-04, 2.98240489e-04, 3.05576036e-04, ...,
        1.30852706e-05, 1.54943269e-05, 1.42022463e-05],
       ...,
       [3.54225564e+00, 4.56530822e+00, 1.56092203e+00, ...,
        1.42152810e-06, 1.80009906e-06, 2.22559408e-06],
       [3.32987604e-04, 3.46781713e-04, 3.43996369e-04, ...,
        2.56632663e-04, 2.25475732e-04, 1.57156218e-04],
       [1.58050651e-03, 1.09680526e-02, 3.50184347e-02, ...,
        3.05826028e-06, 2.91710374e-06, 2.22420323e-06]])

In [ ]:
gaus_ = GaussianNB()
params = {'var_smoothing': np.logspace(0,-9, num=100)}
clf = GridSearchCV(gaus_, params)
clf.fit(train_x, train_y)



GridSearchCV(cv=None, error_score=nan,
             estimator=GaussianNB(priors=None, var_smoothing=1e-09),
             iid='deprecated', n_jobs=None,
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02,...
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
from sklearn.metrics import accuracy_score
results_df = pd.DataFrame(clf.cv_results_)
results_df = results_df.sort_values(by=['rank_test_score'])
results_df = (results_df .set_index(results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))).rename_axis('var_smoothing'))
#results_df[['params', 'rank_test_score', 'mean_test_score', 'std_test_score']]
acc= [0 for i in range(100)]
expected = test_y
for i in range(len(results_df)):
  gaus_ = GaussianNB(var_smoothing = results_df["params"][i]["var_smoothing"])
  gaus_.fit(train_x, train_y)
  predicted = gaus_.predict(test_x)
  acc[i] = accuracy_score(expected, predicted)

results_df['accuracy'] = acc
results_df[["params","accuracy"]]

,params,accuracy
var_smoothing,,
2.310129700083158e-09,{'var_smoothing': 2.310129700083158e-09},0.390681
2.848035868435805e-09,{'var_smoothing': 2.848035868435805e-09},0.390681
1.873817422860387e-09,{'var_smoothing': 1.873817422860387e-09},0.390681
1e-09,{'var_smoothing': 1e-09},0.399642
1.2328467394420635e-09,{'var_smoothing': 1.2328467394420635e-09},0.397849
...,...,...
0.43287612810830584,{'var_smoothing': 0.43287612810830584},0.154122
0.533669923120631,{'var_smoothing': 0.533669923120631},0.145161
0.657933224657568,{'var_smoothing': 0.657933224657568},0.141577


In [ ]:
 gaus_ = GaussianNB(var_smoothing = 2.310129700083158e-09)
gaus_.fit(train_x, train_y)
expected = test_y
predicted = gaus_.predict(test_x)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

                 precision    recall  f1-score   support

Acoustic_guitar       0.44      0.17      0.24        66
       Applause       0.46      0.38      0.41        64
      Bass_drum       0.42      0.75      0.54        48
          Cello       0.19      0.20      0.20        59
    Double_bass       0.34      0.75      0.46        71
           Fart       0.44      0.51      0.47        69
          Flute       0.65      0.21      0.31        63
        Shatter       0.57      0.39      0.46        59
        Trumpet       0.30      0.19      0.23        59

       accuracy                           0.39       558
      macro avg       0.42      0.39      0.37       558
   weighted avg       0.42      0.39      0.37       558

[[11  0 12 11 19  9  2  0  2]
 [ 1 24  7 17  4  6  1  1  3]
 [ 0  0 36  0 11  0  0  1  0]
 [ 3  1  2 12 35  1  2  0  3]
 [ 4  0  6  2 53  2  1  2  1]
 [ 0  3 10  4  4 35  1  8  4]
 [ 5  8  8  7  7  2 13  0 13]
 [ 0  3  1  5  6 21  0 23  0]
 [ 1 13  3  5 18